# Ініціалізація проекту

In [ ]:
import pprint # не забыть удалить

In [37]:
!pip install urllib
!pip install requests
!pip install fuzzywuzzy
!pip install python-Levenshtein

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
     |████████████████████████████████| 50 kB 3.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=b20f86c811ca66b3779b5d4ce2c7937ec9b030b858a1626c25e598271f31f164
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import sys
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/Study/DevNet/HikkaShop']

In [7]:
sys.path.append("/content/drive/MyDrive/Study/DevNet/HikkaShop")

# Модуль обробки природної мови (NLP)

## Використання Waifu AI Api

In [9]:
import requests as re
import urllib.parse
import Keys

def get_response(from_name, to_name, situation, user_id, message):
  data_params = {
    "from_name": from_name,
    "to_name": to_name,
    "situation": situation,
    "message": message,

    "user_id": user_id,
    "preset_mode": "waifu",
    "translate_from": "uk",
  }

  def strTransform(params):
      res = ""
      for param, value in params.items():
          res += param+"="+urllib.parse.quote(value)+"&"
      return res[:len(res)-1]

  params = {
      "url": "https://waifu-ai.p.rapidapi.com/path",
      "headers": {
          'content-type': 'application/x-www-form-urlencoded',
          'x-rapidapi-host': 'waifu-ai.p.rapidapi.com',
          'x-rapidapi-key': Keys.keyMax
      },
      "data": strTransform(data_params)
  }
  
  response = re.post(**params)
  return response.text

## Створення логфайлу бесіди

In [10]:
def logOutput(log):
  from datetime import datetime
  now = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")
  with open("logfiles/"+now+".txt", "a", encoding="UTF-8") as ouf:
    ouf.write(log)

## Генерація user_id

In [ ]:
import string
import random
alnum = string.ascii_letters + string.digits
user_id = ''.join(random.choice(alnum) for i in range(64))
print(user_id)

'N2UkvFxvvq7Rb0RzZKmeAGsIi0x1zAUfic1UleLNpcSCFZLAbXIDxS9DBXRccevN'

In [11]:
user_id = "N2UkvFxvvq7Rb0RzZKmeAGsIi0x1zAUfic1UleLNpcSCFZLAbXIDxS9DBXRccevN"

In [15]:
class WaifuAI(object):
  def __init__(self, situation: list):
    self.situation = situation

class Customer(WaifuAI):
  def __init__(self, name: str, cash: float):
    self.name = name
    self.cash = cash
    super().situation.append(f"{name} має при собі {cash} гривень.")

  def purchase(self, seller: WaifuAI, merch):
    if merch is None:
      super().situation.append(f"{self.name} намагався купити товар, але {seller.name} не зрозуміла що це за товар.")

    merch_qnty = seller.stock[merch][0]
    merch_price = seller.stock[merch][1]
    
    if merch_qnty == 0:
      super().situation.appned(f"{self.name} намагався купити {merch}, але {merch} закінчився.")
      return

    if cash < merch_price:
      super().situation.appned(f"{self.name} намагався купити {merch}, але {self.name} не має стільки грошей.")
      return

    seller.stock[merch][0] -= 1
    self.cash -= merch_price

class Seller(WaifuAI):
  def __init__(self, name: str, stock: dict):
    self.name = name
    self.stock = stock
    stock = ""
    for merch, v in self.stock.items():
      qnty = v[0]
      price = v[1]
      super().situation.append(f"{name} може продати {merch} у кількості {qnty} за ціною {price} гривень.")
  
  def get_order(self, message, tolerance=70):
    import regex as re
    from fuzzywuzzy import fuzz

    find_merch = re.compile("купити ([\w\s]*).")
    finded_merch = find_merch.search(message)
    if finded_merch:
      max_ratio = 0
      merch = None
      for merch in self.stock.keys():
        ratio = fuzz.ratio(finded_merch, merch)

        max_ratio = ratio if ratio > max_ratio or max_ratio == 0 else max_ratio
        merch = merch if ratio > max_ratio or max_ratio == 0 else merch

      if max_ratio >= tolerance and merch is not None:
        return merch

## Вхідні дані розмови

In [14]:
from merch_list import merch_list

#@title Ствоерення діючих осіб{ run: "auto", form-width: "50%" }
#@markdown Покупець
customer_name = "Тарас-кун" #@param {type: "raw"}
cash =  12.3#@param {type:"number"}

#@markdown Продавець
seller_name =   "Акено-тян" #@param {type: "raw"}
situation = "Акено-тян продає аніме-товари в магазині." #@param {type: "raw"}

WaifuAI.situation = [situation]

costumer = Customer(customer_name, cash)
seller = Seller(seller_name, merch_list)

{'cash': 12.3, 'name': 'Тарас-кун'}


## Надсилання повідомлень

In [ ]:
log = f"""from_name = {from_name}
to_name   = {to_name}
situation = {ext_situation}
user_id   = {user_id}
---\n
"""

while(True):
  message = input(f"{from_name}: ")
  log += f"{from_name}: {message}\n"




  response = get_response(
    from_name = from_name,
    to_name   = to_name,
    situation = ext_situation,
    user_id   = user_id,
    message   = message
  )

  print(f'{to_name}: {response}')
  log += f'{to_name}: {response}\n'

  if (message == "Пока!"):
    logOutput(log)
    break

Акено-тян: Вау! Хто ти?! Ви більше не повинні показувати тут своє обличчя!!
Акено-тян: Так, привіт!
Акено-тян: Пам&#39;ятати тебе? У жодному разі. Це було три роки тому чи щось таке.
Акено-тян: Гм, дайте мені побачити... це не було нічого важливого...
Акено-тян: так. Чому?
Акено-тян: Добре, добре. Гм, чому б тобі не прийти сюди і не дізнатися?
